# LCA with only background

This notebook illustrates how to run LCA with only background using this library.

### Preparation

1. Navigate to the project that contains the ecoinvent database.

In [1]:
import bw2data as bd
import bw2io as bi

bd.projects.set_current("advlca25")

# NOTICE: If you don't have the ecoinvent database, please uncomment this code snippet, fill in the correct ecoinvent database and account information, and then run the cell. The import may take some time.
# bi.import_ecoinvent_release(
#     version='<ecoinvent version>',
#     system_model='consequential',
#     username ='XXX', # use your own
#     password='XXX' # use your own
# )

bd.databases # There should be two databases imported, one includes all inventory data (e.g., ecoinvent-3.11-consequential), the other includes all emission data (e.g., ecoinvent-3.11-biosphere).

Databases dictionary with 4 object(s):
	ALIGNED-biob-prod-dummy
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential
	exldb

2. Import required libraries

In [2]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bamboo_lca.background_importer import *
from bamboo_lca.foreground_importer import *
from bamboo_lca.datapackage_builder import *
from bamboo_lca.uncertainty_handler import *
from bamboo_lca.lca_wrapper import *
from bamboo_lca.uncertainty_importer import *

3. Define required constants

In [3]:
# This is the biosphere emissions.
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - fossil - air",
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

METHOD = ('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global temperature change potential (GTP100)')

# BACKGROUND DATABASE FILE PATH
EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/S.txt")

# FOREGROUND DATABASE FILE PATH
FOREGROUND_1 = os.path.join(os.getcwd(), "data/foreground_system_1.csv")
FOREGROUND_2 = os.path.join(os.getcwd(), "data/foreground_system_2.csv")

# CHARACTERIZATION FACTOR MAPPING FILE PATH
CF_MAPPING_FILE = os.path.join(os.getcwd(), "data/cf_mapping_file.csv")

### Step 1: Get the background database matrices

1. Get technosphere matrix

In [4]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.build_tech_matrix(raw_tech)

print(f"The shape of technosphere matrix is: {tech_matrix.shape}")

The shape of technosphere matrix is: (76, 76)


2. Get biosphere matrix

In [5]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.build_bio_matrix(bio_df, GHG)

print(f"The shape of biosphere matrix is: {bio_matrix.shape}")

The shape of biosphere matrix is: (18, 76)


3. Get characterization factor matrix  
The characterization factors can be extracted from the ecoinvent biosphere database or from `CF_MAPPING_FILE` file.

- If you already have required characterization factors, then you just need to add the characterization factors to the `CFs` column of `CF_MAPPING_FILE` file. Then, run Option 1.
- Otherwise, you need to extract characterization factors from the ecoinvent biosphere database. In this case, you must have the ecoinvent databases imported into Brightway. Then, run Option 2.

Option1: Get characterization factor matrix from the file directly.

In [6]:
# get characterization factor matrix from the file directly.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG)  # By default, source="cf", so you don't need to pass this parameter to the function.

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.00e+00 1.00e+00 1.00e+00 1.00e+00 2.70e+01 2.98e+01 2.98e+01 2.98e+01
 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.73e+02
 2.73e+02 2.52e+04]


Option2: Get characterization factor matrix from Brightway.

In [7]:
# get characterization factor matrix from code.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG, "ecoinvent-3.11-biosphere", METHOD, source="code")

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 4.30000000e+00 5.70000000e+00 5.70000000e+00 5.70000000e+00
 5.70000000e+00 5.70000000e+00 5.70000000e+00 5.70000000e+00
 5.70000000e+00 5.70000000e+00 5.70000000e+00 2.34200000e+02
 2.34200000e+02 2.82148093e+04]


### Step 2: Buiild datapackage

4. Build datapackage

In [8]:
dp_builder = DatapackageBuilder()
datapackage_data = dp_builder.prepare_dp_matrix(tech_matrix, bio_matrix, cf_matrix)
datapackage = dp_builder.prepare_datapackage(datapackage_data)

### Step 3: Run LCA

5. Run static simulation through brightway directly

In [ ]:
# Define the index of the functional unit
mysector = 68  # RoW-Services

bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")

lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[datapackage],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, bg_activities[mysector]}")


Brightway calculated lca score: (236835.42529875704, 'RoW-Services')


### Step 6: Compare to manually LCA

6. Compare with lca manually.

In [10]:
lca_wrapper = LCAWrapper()

full_tech_matrix_manual = tech_matrix.copy()

manual_tech_Data = -full_tech_matrix_manual
np.fill_diagonal(manual_tech_Data, -manual_tech_Data.diagonal())

manual_lca = lca_wrapper.manual_lca(manual_tech_Data, bio_matrix, cf_matrix, mysector)

print(f"Manually calculated lca score: {bg_activities[mysector]}: {manual_lca}")


Manually calculated lca score: RoW-Services: 236835.42529875695
